In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest
import os
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
df = pd.read_csv("All measurements V2, cleaned/all_measurements_correct_30,000.csv")

In [3]:
df

,value,force_vector,timestamp,robot_tcp.tcp_offset,robot_tcp.flange,robot_tcp.velocity_linear,robot_tcp.velocity_angular,robot_tcp.timestamp,robot_tcp.joint_angles,version,...,start_time,end_time,sampling_rate,weight_filter,zero_offset,expected_weight,robot_type,force_sensor_offset,measurement_id,run_index
0,0.324,"[0.775, -0.453, 20.0]",0.000000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...",1.758967e+09,"[0.7639275193214417, -0.9689134520343323, -4.1...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
1,0.324,"[0.785, -0.431, 20.0]",0.001000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...",1.758967e+09,"[0.7639275193214417, -0.9689134520343323, -4.1...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
2,0.325,"[0.8, -0.402, 20.0]",0.002000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...",1.758967e+09,"[0.768875777721405, -1.0236349266818543, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
3,0.325,"[0.81, -0.368, 20.0]",0.003000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...",1.758967e+09,"[0.768875777721405, -1.0236349266818543, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
4,0.326,"[0.815, -0.338, 19.0]",0.003999,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37108150124549866, -0.5987440943717957, 0...","[0.156813803573975, -0.048874791685071466, 0.8...","[-0.02211244230129169, 0.016721851482539372, -...",1.758967e+09,"[0.7692531943321228, -1.026842550640442, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15179139,0.399,"[0.19, 0.668, 16126.0]",0.495248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.759916e+09,1000.0,NaN,-0.168018,0.482,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-08_11-30-00-234675_measure_weight_FORC...,30973
15179140,0.400,"[0.23, 0.691, 16322.0]",0.496248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.759916e+09,1000.0,NaN,-0.168018,0.482,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-08_11-30-00-234675_measure_weight_FORC...,30973
15179141,0.402,"[0.26, 0.711, 16660.0]",0.497248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5301331877708435, -0.18638724088668823, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.75

In [4]:
df.keys()

Index(['value', 'force_vector', 'timestamp', 'robot_tcp.tcp_offset',
       'robot_tcp.flange', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.timestamp',
       'robot_tcp.joint_angles', 'version', 'type', 'start_time', 'end_time',
       'sampling_rate', 'weight_filter', 'zero_offset', 'expected_weight',
       'robot_type', 'force_sensor_offset', 'measurement_id', 'run_index'],
      dtype='object')

In [5]:
df = df.drop('start_time', axis=1)
df = df.drop('end_time', axis=1)
df = df.drop('version', axis=1)
df = df.drop('timestamp', axis=1)
df = df.drop('type', axis=1)
df = df.drop('sampling_rate', axis=1)
df = df.drop('weight_filter', axis=1)
df = df.drop('robot_type', axis=1)
df = df.drop('measurement_id', axis=1)
df = df.drop('robot_tcp.timestamp', axis=1)
df = df.drop('force_sensor_offset', axis=1) # always 1, so contains no information.
df = df.drop('robot_tcp.tcp_offset', axis=1) # always 1, so contains no information.

df['force_sensor_value'] = df['value'] - df['zero_offset'] # does the zero_offset only impact the 'value' colummn? My assumption is yes, but not 100% sure.
df = df.drop('value', axis=1)
df = df.drop('zero_offset', axis=1)

In [6]:
df.keys()

Index(['force_vector', 'robot_tcp.flange', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.joint_angles',
       'expected_weight', 'run_index', 'force_sensor_value'],
      dtype='object')

In [7]:
# df.to_csv('all_measurements_correct_30,000_cleaned', index=False)

In [8]:
# df = pd.read_csv('all_measurements_correct_30,000_cleaned')

In [9]:
df

,force_vector,robot_tcp.flange,robot_tcp.velocity_linear,robot_tcp.velocity_angular,robot_tcp.joint_angles,expected_weight,run_index,force_sensor_value
0,"[0.775, -0.453, 20.0]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...","[0.7639275193214417, -0.9689134520343323, -4.1...",0.335,1,0.340568
1,"[0.785, -0.431, 20.0]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...","[0.7639275193214417, -0.9689134520343323, -4.1...",0.335,1,0.340568
2,"[0.8, -0.402, 20.0]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...","[0.768875777721405, -1.0236349266818543, -4.13...",0.335,1,0.341568
3,"[0.81, -0.368, 20.0]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...","[0.768875777721405, -1.0236349266818543, -4.13...",0.335,1,0.341568
4,"[0.815, -0.338, 19.0]","[[-0.37108150124549866, -0.5987440943717957, 0...","[0.156813803573975, -0.048874791685071466, 0.8...","[-0.02211244230129169, 0.016721851482539372, -...","[0.7692531943321228, -1.026842550640442, -4.13...",0.335,1,0.342568
...,...,...,...,...,...,...,...,...
15179139,"[0.19, 0.668, 16126.0]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...","[3.795844793319702, -1.3299107414535065, -2.00...",0.482,30973,0.567018
15179140,"[0.23, 0.691, 16322.0]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...","[3.795844793319702, -1.3299107414535065, -2.00...",0.482,30973,0.568018
15179141,"[0.26, 0.711, 16660.0]","[[-0.5301331877708435, -0.18638724088668823, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...","[3.795844793319702, -1.3299107414535065, -2.00...",0.482,30973,0.570018
15179142,"[0.284, 0.736, 16932.0]","[[-0.5301331877708435, -0.18638724088668823, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...","[3.795844793319702, -1.3299107414535065, -2.00...",0.482,30973,0.571018


In [10]:
for col in df.columns:
    s = df[col]
    if s.dtype == object and s.str.startswith('[').all() and ~s.str.contains(r'\[\[').all():
        parts = s.str[1:-1].str.split(',', expand=True)        # vectorized split
        parts = parts.apply(pd.to_numeric, errors='coerce')     # vectorized cast
        parts.columns = [f"{col}_{i+1}" for i in range(parts.shape[1])]
        df[parts.columns] = parts
        df.drop(columns=[col], inplace=True)

In [11]:
df

,robot_tcp.flange,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3,robot_tcp.velocity_angular_1,robot_tcp.velocity_angular_2,robot_tcp.velocity_angular_3,robot_tcp.joint_angles_1,robot_tcp.joint_angles_2,robot_tcp.joint_angles_3,robot_tcp.joint_angles_4,robot_tcp.joint_angles_5,robot_tcp.joint_angles_6
0,"[[-0.37170541286468506, -0.5985541343688965, 0...",0.335,1,0.340568,0.775,-0.453,20.0,0.079766,-0.010370,0.902352,0.000027,-0.005977,0.000873,0.763928,-0.968913,-4.142107,-2.721707,4.688728,-0.142706
1,"[[-0.37170541286468506, -0.5985541343688965, 0...",0.335,1,0.340568,0.785,-0.431,20.0,0.079766,-0.010370,0.902352,0.000027,-0.005977,0.000873,0.763928,-0.968913,-4.142107,-2.721707,4.688728,-0.142706
2,"[[-0.3713288903236389, -0.5986536741256714, 0....",0.335,1,0.341568,0.800,-0.402,20.0,0.153676,-0.044164,0.886970,-0.016157,0.011443,-0.002004,0.768876,-1.023635,-4.138558,-2.670519,4.689290,-0.137949
3,"[[-0.3713288903236389, -0.5986536741256714, 0....",0.335,1,0.341568,0.810,-0.368,20.0,0.153676,-0.044164,0.886970,-0.016157,0.011443,-0.002004,0.768876,-1.023635,-4.138558,-2.670519,4.689290,-0.137949
4,"[[-0.37108150124549866, -0.5987440943717957, 0...",0.335,1,0.342568,0.815,-0.338,19.0,0.156814,-0.048875,0.886929,-0.022112,0.016722,-0.001132,0.769253,-1.026843,-4.138413,-2.667505,4.689307,-0.137588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15179139,"[[-0.5305851101875305, -0.18714310228824615, 0...",0.482,30973,0.567018,0.190,0.668,16126.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179140,"[[-0.5305851101875305, -0.18714310228824615, 0...",0.482,30973,0.568018,0.230,0.691,16322.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179141,"[[-0.5301331877708435, -0.18638724088668823, 0...",0.482,30973,0.570018,0.260,0.711,16660.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179142,"[[-0.5301331877708435, -0.18638724088668823, 0...",0.482,30973,0.571018,0.284,0.736,16932.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333


In [12]:
df['robot_tcp.flange'][0]

'[[-0.37170541286468506, -0.5985541343688965, 0.06889799237251282], [0.3220159113407135, 0.9465886354446411, 0.014908394776284695, -0.007313983514904976]]'

In [13]:
col = 'robot_tcp.flange'

s = df[col]

# Clean like before (flatten and remove brackets/spaces)
inner = (
    s.astype(str)
     .str.replace('],', ',', regex=False)
     .str.replace('[', '', regex=False)
     .str.replace(']', '', regex=False)
     .str.replace(' ', '', regex=False)
     .str.replace('\t', '', regex=False)
     .str.strip(',')
)

# Parse each row individually and count numeric tokens
counts = inner.map(lambda x: len(np.fromstring(x, sep=',')))
commas = inner.str.count(',')
expected = commas.mode()[0] + 1 if not commas.mode().empty else None

# Find bad rows
bad_mask = (expected is not None) & ((counts != expected) | (counts == 0))
bad_rows = df.index[bad_mask].tolist()

if bad_rows:
    print(f"⚠️ Found {len(bad_rows)} malformed rows in '{col}' (expected {expected} numbers per row):\n")
    for i in bad_rows[:10]:  # show up to 10 samples
        print(f"Row {i}: {df.at[i, col]}")
else:
    print(f"✅ All rows in '{col}' parsed cleanly ({expected} numbers per row).")

✅ All rows in 'robot_tcp.flange' parsed cleanly (7 numbers per row).


In [14]:
df

,robot_tcp.flange,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3,robot_tcp.velocity_angular_1,robot_tcp.velocity_angular_2,robot_tcp.velocity_angular_3,robot_tcp.joint_angles_1,robot_tcp.joint_angles_2,robot_tcp.joint_angles_3,robot_tcp.joint_angles_4,robot_tcp.joint_angles_5,robot_tcp.joint_angles_6
0,"[[-0.37170541286468506, -0.5985541343688965, 0...",0.335,1,0.340568,0.775,-0.453,20.0,0.079766,-0.010370,0.902352,0.000027,-0.005977,0.000873,0.763928,-0.968913,-4.142107,-2.721707,4.688728,-0.142706
1,"[[-0.37170541286468506, -0.5985541343688965, 0...",0.335,1,0.340568,0.785,-0.431,20.0,0.079766,-0.010370,0.902352,0.000027,-0.005977,0.000873,0.763928,-0.968913,-4.142107,-2.721707,4.688728,-0.142706
2,"[[-0.3713288903236389, -0.5986536741256714, 0....",0.335,1,0.341568,0.800,-0.402,20.0,0.153676,-0.044164,0.886970,-0.016157,0.011443,-0.002004,0.768876,-1.023635,-4.138558,-2.670519,4.689290,-0.137949
3,"[[-0.3713288903236389, -0.5986536741256714, 0....",0.335,1,0.341568,0.810,-0.368,20.0,0.153676,-0.044164,0.886970,-0.016157,0.011443,-0.002004,0.768876,-1.023635,-4.138558,-2.670519,4.689290,-0.137949
4,"[[-0.37108150124549866, -0.5987440943717957, 0...",0.335,1,0.342568,0.815,-0.338,19.0,0.156814,-0.048875,0.886929,-0.022112,0.016722,-0.001132,0.769253,-1.026843,-4.138413,-2.667505,4.689307,-0.137588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15179139,"[[-0.5305851101875305, -0.18714310228824615, 0...",0.482,30973,0.567018,0.190,0.668,16126.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179140,"[[-0.5305851101875305, -0.18714310228824615, 0...",0.482,30973,0.568018,0.230,0.691,16322.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179141,"[[-0.5301331877708435, -0.18638724088668823, 0...",0.482,30973,0.570018,0.260,0.711,16660.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333
15179142,"[[-0.5301331877708435, -0.18638724088668823, 0...",0.482,30973,0.571018,0.284,0.736,16932.0,0.226613,0.411088,0.686115,-0.018644,-0.041923,-0.001337,3.795845,-1.329911,-2.008778,-1.369907,1.565777,4.407333


In [17]:
col = "robot_tcp.flange"
chunk = 1_000_000

s = (df[col].astype(str)
       .str.replace(r'\],\s*\[', ',', regex=True)
       .str.replace(r'[\[\]\s]', '', regex=True)
       .str.replace(',,', ',', regex=False)
       .str.strip(','))

w = s.iloc[0].count(',') + 1
out = np.empty((len(df), w), float)

for start in range(0, len(df), chunk):
    end = min(start + chunk, len(df))
    block = s.iloc[start:end].to_numpy(copy=False)
    big = '\n'.join(block.tolist())
    arr = np.fromstring(big, sep=',', dtype=float)
    if arr.size != (end - start) * w:
        arr = np.concatenate([np.fromstring(x, sep=',', dtype=float) for x in block])
    out[start:end] = arr.reshape(end - start, w)

df[[f"{col}_{i+1}" for i in range(w)]] = out
df.drop(columns=[col], inplace=True)

C:\Users\piete\AppData\Local\Temp\ipykernel_8220\787671431.py:17: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  arr = np.fromstring(big, sep=',', dtype=float)


In [18]:
df

,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3,robot_tcp.velocity_angular_1,...,robot_tcp.joint_angles_4,robot_tcp.joint_angles_5,robot_tcp.joint_angles_6,robot_tcp.flange_1,robot_tcp.flange_2,robot_tcp.flange_3,robot_tcp.flange_4,robot_tcp.flange_5,robot_tcp.flange_6,robot_tcp.flange_7
0,0.335,1,0.340568,0.775,-0.453,20.0,0.079766,-0.010370,0.902352,0.000027,...,-2.721707,4.688728,-0.142706,-0.371705,-0.598554,0.068898,0.322016,0.946589,0.014908,-0.007314
1,0.335,1,0.340568,0.785,-0.431,20.0,0.079766,-0.010370,0.902352,0.000027,...,-2.721707,4.688728,-0.142706,-0.371705,-0.598554,0.068898,0.322016,0.946589,0.014908,-0.007314
2,0.335,1,0.341568,0.800,-0.402,20.0,0.153676,-0.044164,0.886970,-0.016157,...,-2.670519,4.689290,-0.137949,-0.371329,-0.598654,0.070673,0.322003,0.946594,0.014877,-0.007324
3,0.335,1,0.341568,0.810,-0.368,20.0,0.153676,-0.044164,0.886970,-0.016157,...,-2.670519,4.689290,-0.137949,-0.371329,-0.598654,0.070673,0.322003,0.946594,0.014877,-0.007324
4,0.335,1,0.342568,0.815,-0.338,19.0,0.156814,-0.048875,0.886929,-0.022112,...,-2.667505,4.689307,-0.137588,-0.371082,-0.598744,0.072457,0.322000,0.946595,0.014853,-0.007342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15179139,0.482,30973,0.567018,0.190,0.668,16126.0,0.226613,0.411088,0.686115,-0.018644,...,-1.369907,1.565777,4.407333,-0.530585,-0.187143,0.548930,-0.462112,0.886810,0.003469,-0.002924
15179140,0.482,30973,0.568018,0.230,0.691,16322.0,0.226613,0.411088,0.686115,-0.018644,...,-1.369907,1.565777,4.407333,-0.530585,-0.187143,0.548930,-0.462112,0.886810,0.003469,-0.002924
15179141,0.482,30973,0.570018,0.260,0.711,16660.0,0.226613,0.411088,0.686115,-0.018644,...,-1.369907,1.565777,4.407333,-0.530133,-0.186387,0.550307,-0.462154,0.886788,0.003433,-0.002924
15179142,0.482,30973,0.571018,0.284,0.736,16932.0,0.226613,0.411088,0.686115,-0.018644,...,-1.369907,1.565777,4.407333,-0.530133,-0.186387,0.550307,-0.462154,0.886788,0.003433,-0.002924


In [20]:
df.to_csv('all_measurements_summery_30,000_cleaned.csv', index=False)